### MPI Tutorial


In [1]:
## Imports
import ipyparallel as ipp
import dolfinx     as dfx

from mpi4py import MPI


In [2]:
cluster = ipp.Cluster(engines = "mpi", n = 2)
rc = cluster.start_and_connect_sync()

Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1681374962-bxss:Starting 2 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/2 [00:00<?, ?engine/s]

In [3]:
dv = rc[:]

#### Running two processes in parallel with the %%px magic command.
When using MPI a communicator must be specified. COMM_WORLD is a global communicator that contains all processes that the local rank can communicate with at initialization. COMM_SELF is a communicator local to a process.

In [4]:
%%px
comm = MPI.COMM_WORLD # MPI communicator
# Print Hello World from all processors
print(f"Hello World from rank {comm.rank} of {comm.size}")

[stdout:0] Hello World from rank 0 of 2


[stdout:1] Hello World from rank 1 of 2


#### Communication between two processors.

In [5]:
%%px 
comm = MPI.COMM_WORLD # MPI communicator

if comm.rank == 0:
    # Send num1 from processor 1 to 2
    num1 = 5
    comm.send(num1, dest = 1, tag = 1)
    print(f"Rank {comm.rank}, num1 = {num1}")
    
elif comm.rank == 1:
    # Receive num1 from processor 1, add num3 = num1 + num2 and print num3
    num2 = 3
    num1 = comm.recv(source = 0, tag = 1)
    num3 = num1 + num2
    print(f"Rank {comm.rank}, num3 = num1 + num2 = {num3}")

[stdout:0] Rank 0, num1 = 5


[stdout:1] Rank 1, num3 = num1 + num2 = 8


#### MPI communication in DOLFINx with mpi4py.
When constructing a mesh in DOLFINx, the type of communicator must be specified. The mesh is partitioned and distributed over different processes.

A parameter 'ghost_mode' must be specified. This determines how 

In [6]:
%%px
import dolfinx as dfx
import dolfinx.io
comm = MPI.COMM_WORLD # MPI communicator

Nx, Ny = 2, 2 # Mesh size

# Create a unit square mesh
mesh = dfx.mesh.create_unit_square(comm, Nx, Ny, ghost_mode = dfx.cpp.mesh.GhostMode.shared_facet)
mesh.topology.create_connectivity(0, 2)
mesh.topology.create_connectivity(1, 2)
mesh.topology.create_connectivity(2, 1)
mesh.topology.create_connectivity(2, 0)
mesh.topology.create_connectivity(0, 1)
mesh.topology.create_connectivity(1, 0)

def mpi_print(s):
    print(f"Rank {comm.rank}: {s}")

mpi_print(f"Number of local cells: {mesh.topology.index_map(2).size_local}")
mpi_print(f"Number of global cells: {mesh.topology.index_map(2).size_global}")
print("Cell (dim = 2) to vertex (dim = 0) connectivity:")
mpi_print(mesh.topology.connectivity(2, 0))


if comm.size == 1:
    print("Cell (dim = 2) to facet (dim = 1) connectivity:")
    mpi_print(mesh.topology.connectivity(2, 1))

if comm.size == 2:
    mpi_print(f"Ghost cells (global numbering): {mesh.topology.index_map(2).ghosts}")
    #mpi_print(f"Ghost owner rank: {mesh.topology.index_map(2).ghost_owner_rank()}")

          

[stdout:1] Rank 1: Number of local cells: 4
Rank 1: Number of global cells: 8
Cell (dim = 2) to vertex (dim = 0) connectivity:
Rank 1: <AdjacencyList> with 6 nodes
  0: [5 0 1 ]
  1: [0 1 2 ]
  2: [0 3 2 ]
  3: [1 2 4 ]
  4: [5 6 1 ]
  5: [1 7 4 ]

Rank 1: Ghost cells (global numbering): [0 3]


[stdout:0] Rank 0: Number of local cells: 4
Rank 0: Number of global cells: 8
Cell (dim = 2) to vertex (dim = 0) connectivity:
Rank 0: <AdjacencyList> with 6 nodes
  0: [0 1 4 ]
  1: [1 4 2 ]
  2: [1 3 2 ]
  3: [4 2 5 ]
  4: [0 6 4 ]
  5: [4 7 5 ]

Rank 0: Ghost cells (global numbering): [4 7]


#### Dolfinx function spaces.
The degrees of freedom of a function space in dolfinx is distributed over the nodes of the mesh.

In [7]:
%%px
import dolfinx as dfx

comm = MPI.COMM_WORLD # MPI communicator

Nx, Ny = 2, 2 # Mesh size

# Create a unit square mesh
mesh = dfx.mesh.create_unit_square(comm, Nx, Ny, ghost_mode = dfx.cpp.mesh.GhostMode.shared_facet)
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

print(f"Global dofmap size: {V.dofmap.index_map.size_global}")
print(f"Local dofmap size: {V.dofmap.index_map.size_local}")
print(f"Ghosts: {V.dofmap.index_map.ghosts}")

[stdout:1] Global dofmap size: 9
Local dofmap size: 5
Ghosts: [0 1 2]


[stdout:0] Global dofmap size: 9
Local dofmap size: 4
Ghosts: [5 8 4 6]


#### Dolfinx functions.
The degrees of freedom of a dolfinx function is distributed over the nodes of the mesh. PETSc vector represantations of dolfinx functions have the attribute .localForm(), which can be used to access a local array with space for both owned and local degrees of freedom.

In [8]:
%%px

comm = MPI.COMM_WORLD # MPI communicator

Nx, Ny = 2, 2 # Mesh size

# Create a unit square mesh
mesh = dfx.mesh.create_unit_square(comm, Nx, Ny, ghost_mode = dfx.cpp.mesh.GhostMode.shared_facet)
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))
u = dfx.fem.Function(V)

print(V.dofmap.index_map.size_local*V.dofmap.index_map_bs, V.dofmap.index_map.num_ghosts*V.dofmap.index_map_bs)
u_vec = u.vector

print(f"Local size of vector: {u_vec.getLocalSize()}")

# .localForm() can be used to access a local array with space for both owned and local degrees of freedom
with u_vec.localForm() as u_local:
    print(f"Local + Ghost size of vector: {u_local.getLocalSize()}")

u_vec.ghostUpdate()

[stdout:1] 5 3
Local size of vector: 5
Local + Ghost size of vector: 8


[stdout:0] 4 4
Local size of vector: 4
Local + Ghost size of vector: 8


#### Assembling scalars, vectors, matrices.
Example: assembling a vector in parallel.

In [9]:
%%px
import ufl
from petsc4py import PETSc
comm = MPI.COMM_WORLD # MPI communicator

Nx, Ny = 2, 1 # Mesh size

# Create a unit square mesh
mesh = dfx.mesh.create_unit_square(comm, Nx, Ny, ghost_mode = dfx.cpp.mesh.GhostMode.none)

# Create a first-order Lagrange finite element space
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction (V)

# UFL form of right-hand side
L = ufl.inner(1.0, v) * ufl.dx
L = dfx.fem.form(L)

# Assemble UFL form into a vector
_b = dfx.fem.Function(V)
dfx.fem.petsc.assemble_vector(_b.vector, L)
_b.x.scatter_forward

# Print the size of the index map and the number of ghost nodes
print(V.dofmap.index_map.size_local*V.dofmap.index_map_bs, V.dofmap.index_map.num_ghosts*V.dofmap.index_map_bs)

print("Prior to communication")
print(f"Rank: {comm.rank}: {_b.x.array}")   
# Add values from ghost regions and accumulate them on the owning process
_b.x.scatter_reverse(dfx.la.ScatterMode.add)

#_b.vector.ghostUpdate(addv = PETSc.InsertMode.ADD, mode = PETSc.ScatterMode.REVERSE)

print("After ADD/REVERSE update")
print(f"Rank: {comm.rank}: {_b.x.array}")   

# Ghost points still not updated, so their values are inconsistent
# Get value from owning process and update the ghosts
_b.x.scatter_forward()
#_b.ghostUpdate(addv = PETSc.InsertMode.INSERT, mode = PETSc.ScatterMode.FORWARD)

print("After INSERT/FORWARD update")
print(f"Rank: {comm.rank}: {_b.x.array}")   


[stdout:0] 3 1
Prior to communication
Rank: 0: [0.16666667 0.08333333 0.08333333 0.16666667]
After ADD/REVERSE update
Rank: 0: [0.16666667 0.25       0.08333333 0.16666667]
After INSERT/FORWARD update
Rank: 0: [0.16666667 0.25       0.08333333 0.25      ]


[stdout:1] 3 1
Prior to communication
Rank: 1: [0.08333333 0.16666667 0.08333333 0.16666667]
After ADD/REVERSE update
Rank: 1: [0.08333333 0.16666667 0.25       0.16666667]
After INSERT/FORWARD update
Rank: 1: [0.08333333 0.16666667 0.25       0.25      ]


### Solving a variational problem
We will consider solving a Poisson problem on the unit square domain, denoted $\Omega$. The strong form of the problem is: determine $u$ such that
\begin{align}
    -\nabla^2 u &= f \quad \mathrm{in} \ \Omega, \\
    u &= g \quad \mathrm{on} \ \partial\Omega,
\end{align}
where $\partial\Omega$ is the boundary of the domain. The weak form of the problem is derived by multiplying the PDE with a test function $v$, integrating over the domain and applying integration by parts. This yields

$$\int_{\Omega} \nabla u \cdot \nabla v dx = \int_{\Omega}f v dx$$

where the boundary integral vanishes because $v = 0$ on the boundary due to the Dirichlet boundary condition. For simplicity we set $g = 0$.

In [25]:
%%px
import ufl
from petsc4py import PETSc
comm = MPI.COMM_WORLD # MPI communicator

Nx, Ny = 2, 1 # Mesh size

# Create a unit square mesh
mesh = dfx.mesh.create_unit_square(comm, Nx, Ny, ghost_mode = dfx.cpp.mesh.GhostMode.none)
mesh.topology.create_entities(mesh.topology.dim - 1)
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

# Create a first-order Lagrange finite element space
V = dfx.fem.FunctionSpace(mesh, ("CG", 1))

# Trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction (V)

u_h = dfx.fem.Function(V) # Solution function

f = dfx.fem.Function(V) # Source term
f.x.set(1) # Set function value to 1


# UFL form of the bilinear form
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
bilinear_form = dfx.fem.form(a)

# UFL form of right-hand side
L = f * v * ufl.dx
linear_form = dfx.fem.form(L)

# Boundary condition function
g = dfx.fem.Function(V) # Dolfinx function, default function value = 0

# Get the dofs of the boundary facets
boundary_facets = dfx.mesh.exterior_facet_indices(mesh.topology)
boundary_dofs   = dfx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, boundary_facets)
bc_g = dfx.fem.dirichletbc(g, boundary_dofs)

bcs = [bc_g]

# Assemble matrix from the bilinear form
A = dfx.fem.petsc.assemble_matrix(bilinear_form, bcs = bcs)
A.assemble()

# Assemble UFL form into a vector
_b = dfx.fem.Function(V) # Dolfinx function of right-hand side
dfx.fem.petsc.assemble_vector(_b.vector, linear_form)
dfx.fem.petsc.apply_lifting(_b.vector, [bilinear_form], bcs = [bcs])
_b.x.scatter_reverse(dfx.la.ScatterMode.add)
dfx.fem.petsc.set_bc(_b.vector, bcs = bcs)

# Create a (direct) linear solver
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType("preonly")
solver.getPC().setType("lu")
solver.getPC().setFactorSolverType("mumps")

# Solve the variational problem
solver.solve(_b.vector, u_h.vector)


print("u_h array:", u_h.x.array)
u_h.x.scatter_reverse(dfx.la.ScatterMode.add)
u_h.x.scatter_forward()

print("u_h array after communication:", u_h.x.array)

# Print the size of the index map and the number of ghost nodes
print(V.dofmap.index_map.size_local*V.dofmap.index_map_bs, V.dofmap.index_map.num_ghosts*V.dofmap.index_map_bs)

print("Prior to communication")
print(f"Rank: {comm.rank}: {_b.x.array}")   
# Add values from ghost regions and accumulate them on the owning process
_b.x.scatter_reverse(dfx.la.ScatterMode.add)

#_b.vector.ghostUpdate(addv = PETSc.InsertMode.ADD, mode = PETSc.ScatterMode.REVERSE)

print("After ADD/REVERSE update")
print(f"Rank: {comm.rank}: {_b.x.array}")   

# Ghost points still not updated, so their values are inconsistent
# Get value from owning process and update the ghosts
_b.x.scatter_forward()
#_b.ghostUpdate(addv = PETSc.InsertMode.INSERT, mode = PETSc.ScatterMode.FORWARD)

print("After INSERT/FORWARD update")
print(f"Rank: {comm.rank}: {_b.x.array}")   


[stdout:1] u_h array: [0. 0. 0. 0.]
u_h array after communication: [0. 0. 0. 0.]
3 1
Prior to communication
Rank: 1: [0.         0.         0.         0.16666667]
After ADD/REVERSE update
Rank: 1: [0.         0.         0.16666667 0.16666667]
After INSERT/FORWARD update
Rank: 1: [0.         0.         0.16666667 0.16666667]


[stdout:0] u_h array: [0. 0. 0. 0.]
u_h array after communication: [0. 0. 0. 0.]
3 1
Prior to communication
Rank: 0: [0.         0.         0.         0.16666667]
After ADD/REVERSE update
Rank: 0: [0.         0.16666667 0.         0.16666667]
After INSERT/FORWARD update
Rank: 0: [0.         0.16666667 0.         0.16666667]
